In [1]:
import gradio as gr
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import os


In [2]:
from keras.models import load_model
facial = load_model('facial_recognition_model.h5')


def facial_prediction(img):
    img = cv2.resize(img, (90, 90))
    labels = {0: 'Angry', 1: 'Fear', 2: 'Happy',
              3: 'Neutral', 4: 'Sad', 5: 'Suprise'}
    color = {0: (0, 0, 255), 1: (1, 1, 1), 2: (0, 255, 0), 3: (
        255, 255, 255), 4: (100, 100, 100), 5: (255, 255, 255)}
    x = []
    x.append(img)
    x = np.array(x)
    pred = facial.predict(x)
    pred = np.argmax(pred)
    return labels[pred], color[pred]


In [3]:
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


In [4]:
# data=[]
def facial_expression():
    capture = cv2.VideoCapture(0)
    while True:
        flag, img = capture.read()
        try:
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces_index = face_classifier.detectMultiScale(
                gray_img, 1.0485258, 6)
            for (x, y, w, h) in faces_index:
                face = img[y:y+h, x:x+w]
                face = cv2.resize(face, (90, 90))
                text, color = facial_prediction(face)
                cv2.rectangle(img, (x, y), (x+w, y+h), color, thickness=4)
                cv2.putText(img, text, (130, 180),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                # data.append(face)
            cv2.imshow('ESC to exit', img)
            if cv2.waitKey(2) == 27:
                break
        except Exception as e:
            print(e)
            break
    cv2.destroyAllWindows()


facial_expression()


1/1 [==============================] - 0s 58ms/step


In [5]:
def classify_image(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_index = face_classifier.detectMultiScale(gray_img, 1.0485258, 6)
    ans="nun"
    for (x, y, w, h) in faces_index:
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (90, 90))
        text, color = facial_prediction(face)
        cv2.rectangle(img, (x, y), (x+w, y+h), color, thickness=4)
        cv2.putText(img, text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    return img


In [9]:
gr.Interface(fn=classify_image,
             inputs=gr.Image(),
             outputs=gr.Image(),
             examples=["sample_image1.jpeg"]
             ).launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 93ms/step
